In [1]:
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import os
import numpy as np
import tensorflow as tf

from evaluations import get_val_data, perform_val
from models import ArcFaceModel
from utils import set_memory_growth, load_yaml

In [2]:
### IJB-C Dataset

# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'CosHead'
# head_type = 'ArcHead'
backbone_type = 'ResNet50' # MobileNetV2, ResNet50, EfficientNetLite0~6
# backbone_type = 'MobileNetV2' # MobileNetV2, ResNet50, EfficientNetLite0~6

# w_decay=5e-4
# num_classes = 85742 
# dataset_len = 5822653 
# base_lr = 0.01 # initially 0.01
# epochs = 20
# save_steps = 1000

# train_size = int(0.8 * dataset_len)
# print(train_size)
# steps_per_epoch = train_size // batch_size
# print(steps_per_epoch)
# val_size = dataset_len - train_size
# print(val_size)
# validation_steps = val_size // batch_size
# print(validation_steps)
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False

dgx = True

In [7]:
from glob import glob
if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    
    if backbone_type == 'MobileNetV2':
        if head_type == 'CosHead':
            save_name = "ms1m_mobilenet_check/Cosface/SGD/*"
        elif head_type == 'ArcHead':
            save_name = "ms1m_mobilenet_check/Arcface/SGD/*"
        else:
            save_name = "ms1m_mobilenet_check/Sphereface/SGD/*"
            
    elif backbone_type == 'ResNet50':
        if head_type == 'CosHead':
            save_name = "ms1m_resnet50_check/Cosface/SGD/*"
        elif head_type == 'ArcHead':
            save_name = "ms1m_resnet50_check/Arcface/SGD/*"
        else:
            save_name = "ms1m_resnet50_check/Sphereface/SGD/*"
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = "ms1m_resnet50/SGD/*"
    
## collect loss in checkpoints
file_list = []
for files in glob(base_dir+save_name):
#     file_list.append(files.split('/')[-1].split('l_')[-1])
    if not files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
        loss = float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] )
    file_list.append( loss  )
file_list.sort()

# print(file_list[0:3])

load_file_name = []
for files in glob(base_dir+save_name):
    if files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
        pass
    elif file_list[0] == float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] ) and files.split('/')[-1].split('l_')[-1].split('.ckpt')[-1]!='.index':
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

0
/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/Cosface/SGD/e_1_l_33.28236389160156.ckpt


In [8]:
# flags.DEFINE_string('cfg_path', './configs/arc_res50.yaml', 'config file path')
# flags.DEFINE_string('gpu', '0', 'which gpu to use')
# flags.DEFINE_string('img_path', '', 'path to input image')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

logger = tf.get_logger()
logger.disabled = True
logger.setLevel(logging.FATAL)
set_memory_growth()

# cfg = load_yaml(FLAGS.cfg_path)

model = ArcFaceModel(size=input_size,
                     backbone_type=backbone_type,
                     training=False)

model.load_weights(best_checkpoint)

# ckpt_path = tf.train.latest_checkpoint('./checkpoints/' + cfg['sub_name'])
# if ckpt_path is not None:
#     print("[*] load ckpt from {}".format(ckpt_path))
#     model.load_weights(ckpt_path)
# else:
#     print("[*] Cannot find ckpt from {}.".format(ckpt_path))
#     exit()

# if FLAGS.img_path:
#     print("[*] Encode {} to ./output_embeds.npy".format(FLAGS.img_path))
#     img = cv2.imread(FLAGS.img_path)
#     img = cv2.resize(img, (cfg['input_size'], cfg['input_size']))
#     img = img.astype(np.float32) / 255.
#     if len(img.shape) == 3:
#         img = np.expand_dims(img, 0)
#     embeds = l2_norm(model(img))
#     np.save('./output_embeds.npy', embeds)

print("[*] Loading LFW, AgeDB30 and CFP-FP...")
lfw, agedb_30, cfp_fp, lfw_issame, agedb_30_issame, cfp_fp_issame = get_val_data("/raid/workspace/honghee/")

print("[*] Perform Evaluation on LFW...")
acc_lfw, best_th = perform_val(
    embd_shape, batch_size, model, lfw, lfw_issame,
    is_ccrop=False)
print("    acc {:.4f}, th: {:.2f}".format(acc_lfw, best_th))

print("[*] Perform Evaluation on AgeDB30...")
acc_agedb30, best_th = perform_val(
    embd_shape, batch_size, model, agedb_30,
    agedb_30_issame, is_ccrop=False)
print("    acc {:.4f}, th: {:.2f}".format(acc_agedb30, best_th))

print("[*] Perform Evaluation on CFP-FP...")
acc_cfp_fp, best_th = perform_val(
    embd_shape, batch_size, model, cfp_fp, cfp_fp_issame,
    is_ccrop=False)
print("    acc {:.4f}, th: {:.2f}".format(acc_cfp_fp, best_th))

  1%|          | 1/94 [00:00<00:16,  5.63it/s]

[*] Loading LFW, AgeDB30 and CFP-FP...
[*] Perform Evaluation on LFW...


  0%|          | 0/94 [00:00<?, ?it/s]

    acc 0.9702, th: 1.23
[*] Perform Evaluation on AgeDB30...


  0%|          | 0/110 [00:00<?, ?it/s]

    acc 0.8398, th: 1.45
[*] Perform Evaluation on CFP-FP...


100%|██████████| 110/110 [00:18<00:00,  5.98it/s]


    acc 0.8476, th: 1.50
